In [1]:
import pandas as pd
import py4cytoscape as p4c
import time
import requests

In [2]:
p4c.__version__

'1.6.0'

In [3]:
p4c.cytoscape_ping()

You are connected to Cytoscape!


'You are connected to Cytoscape!'

In [ ]:
suid = create3Dnetwork("https://raw.githubusercontent.com/ecell/transomics2cytoscape/master/inst/extdata/usecase1/yugi2014.tsv", "yugi2014.xml")

In [94]:
networkLayers = "https://raw.githubusercontent.com/ecell/transomics2cytoscape/master/inst/extdata/usecase1/yugi2014.tsv"

In [96]:
layerTable = pd.read_table(networkLayers, header=None, sep="\t")
networkSUID = layerTable.apply(importLayer, axis=1)

Importing rno04910.xml


c:\Users\kozo2\mambaforge\lib\site-packages\py4cytoscape-1.6.0-py3.10.egg\py4cytoscape\tables.py:502: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  data_subset.dtypes.iteritems()]


Importing rno01100.xml


c:\Users\kozo2\mambaforge\lib\site-packages\py4cytoscape-1.6.0-py3.10.egg\py4cytoscape\tables.py:502: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  data_subset.dtypes.iteritems()]


Creating a node at each midpoint of the edge...
Column "KEGG_EDGE_SUBTYPES" has only 0 elements, but should have 1316
getting node positions...
getting edge information...
creating midpoint nodes...


c:\Users\kozo2\mambaforge\lib\site-packages\py4cytoscape-1.6.0-py3.10.egg\py4cytoscape\tables.py:502: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  data_subset.dtypes.iteritems()]


deleting edges...
start collecting node IDs to connect...
Importing rno01100.xml


c:\Users\kozo2\mambaforge\lib\site-packages\py4cytoscape-1.6.0-py3.10.egg\py4cytoscape\tables.py:502: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  data_subset.dtypes.iteritems()]


In [97]:
layerTable = pd.concat([layerTable, networkSUID], axis=1)
nodetables = layerTable.apply(getNodeTableWithLayerinfo, axis=1)

Column "KEGG_NODE_LABEL_LIST" has only 73 elements, but should have 74
Column "KEGG_NODE_LABEL_LIST" has only 3321 elements, but should have 4637


In [100]:
layeredNodes = getLayeredNodes(nodetables)

In [105]:
nodetables[2]

,SUID,shared name,name,selected,KEGG_NODE_X,KEGG_NODE_Y,KEGG_NODE_WIDTH,KEGG_NODE_HEIGHT,KEGG_NODE_LABEL,KEGG_NODE_LABEL_LIST_FIRST,...,KEGG_NODE_FILL_COLOR,KEGG_NODE_REACTIONID,KEGG_NODE_TYPE,KEGG_NODE_SHAPE,KEGG_LINK,row.names,x_location,y_location,z_location,LAYER_INDEX
57344,57344,path:rno01100:5661,path:rno01100:5661,False,2024,2333,14,14,C02630,2-Hydroxyglutarate,...,#B3B3E6,None,compound,circle,https://www.kegg.jp/dbget-bin/www_bget?C02630,57344,2024,2333,1,metabolite-layer
49154,49154,path:rno01100:7063,path:rno01100:7063,False,4809,1147,14,14,C05366,(+)-Pinoresinol,...,#E0E0E0,None,compound,circle,https://www.kegg.jp/dbget-bin/www_bget?C05366,49154,4809,1147,1,metabolite-layer
57347,57347,path:rno01100:5673,path:rno01100:5673,False,2671,2065,14,14,C00490,Itaconate,...,#B3B3E6,None,compound,circle,https://www.kegg.jp/dbget-bin/www_bget?C00490,57347,2671,2065,1,metabolite-layer
49157,49157,path:rno01100:7062,path:rno01100:7062,False,3584,1724,14,14,C22149,None,...,#E0E0E0,None,compound,circle,https://www.kegg.jp/dbget-bin/www_bget?C22149,49157,3584,1724,1,metabolite-layer
57350,57350,path:rno01100:5678,path:rno01100:5678,False,2698,943,14,14,C00345,6-Phospho-D-gluconate,...,#6C63F6,None,compound,circle,https://www.kegg.jp/dbget-bin/www_bget?C00345,57350,2698,943,1,metabolite-layer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49145,49145,path:rno01100:7066,path:rno01100:7066,False,4809,1321,14,14,C10682,Matairesinol,...,#E0E0E0,None,compound,circle,https://www.kegg.jp/dbget-bin/www_bget?C10682,49145,4809,1321,1,metabolite-layer
57338,57338,path:rno01100:5650,path:rno01100:5650,False,3066,1956,14,14,C21017,None,...,#B3B3E6,None,compound,circle,https://www.kegg.jp/dbget-bin/www_bget?C21017,57338,3066,1956,1,metabolite-layer
49148,49148,path:rno01100:7065,path:rno01100:7065,False,4809,1266,14,14,C18167,None,...,#E0E0E0,None,compound,circle,https://www.kegg.jp/dbget-bin/www_bget?C18167,49148,4809,1266,1,metabolite-layer
57341,57341,path:rno01100:5651,path:rno01100:5651,False,3066,1785,14,14,C21018,None,...,#B3B3E6,None,compound,circle,https://www.kegg.jp/dbget-bin/www_bget?C21018,57341,3066,1785,1,metabolite-layer


In [101]:
nodetable3d = pd.concat(nodetables.array)

,id,shared name,name,selected,KEGG_NODE_X,KEGG_NODE_Y,KEGG_NODE_WIDTH,KEGG_NODE_HEIGHT,KEGG_NODE_LABEL,KEGG_NODE_LABEL_LIST_FIRST,...,KEGG_ID,KEGG_NODE_LABEL_COLOR,KEGG_NODE_FILL_COLOR,KEGG_NODE_REACTIONID,KEGG_NODE_TYPE,KEGG_NODE_SHAPE,KEGG_LINK,row.names,x_location,y_location
258,258,path:rno04910:17,path:rno04910:17,False,642,693,46,17,Mapk1...,Mapk1...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,261,path:rno04910:18,path:rno04910:18,False,956,230,46,17,Fasn,Fasn,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,264,path:rno04910:19,path:rno04910:19,False,875,222,46,17,Prkag1...,Prkag1...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,267,path:rno04910:20,path:rno04910:20,False,981,408,8,8,C00369,Starch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270,270,path:rno04910:21,path:rno04910:21,False,824,510,8,8,C00575,cAMP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49145,49145,path:rno01100:7066,path:rno01100:7066,False,4809,1321,14,14,C10682,Matairesinol,...,[cpd:C10682],#E0E0E0,#E0E0E0,None,compound,circle,https://www.kegg.jp/dbget-bin/www_bget?C10682,49145.0,4809,1321
57338,57338,path:rno01100:5650,path:rno01100:5650,False,3066,1956,14,14,C21017,None,...,[cpd:C21017],#B3B3E6,#B3B3E6,None,compound,circle,https://www.kegg.jp/dbget-bin/www_bget?C21017,57338.0,3066,1956
49148,49148,path:rno01100:7065,path:rno01100:7065,False,4809,1266,14,14,C18167,None,...,[cpd:C18167],#E0E0E0,#E0E0E0,None,compound,circle,https://www.kegg.jp/dbget-bin/www_bget?C18167,49148.0,4809,1266
57341,57341,path:rno01100:5651,path:rno01100:5651,False,3066,1785,14,14,C21018,None,...,[cpd:C21018],#B3B3E6,#B3B3E6,None,compound,circle,https://www.kegg.jp/dbget-bin/www_bget?C21018,57341.0,3066,1785


In [4]:
networkLayers = "https://raw.githubusercontent.com/ecell/transomics2cytoscape/master/inst/extdata/usecase1/yugi2014.tsv"
layerTable = pd.read_table(networkLayers, header=None, sep="\t")

In [5]:
layerTable

,0,1,2,3
0,kinase-layer,rno04910,1500,False
1,enzyme-layer,rno01100,900,True
2,metabolite-layer,rno01100,1,False


In [9]:
networkSUID = layerTable.apply(importLayer, axis=1)

Importing rno04910.xml


c:\Users\fz8348\mambaforge\lib\site-packages\py4cytoscape-1.6.0-py3.10.egg\py4cytoscape\tables.py:502: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


Importing rno01100.xml


c:\Users\fz8348\mambaforge\lib\site-packages\py4cytoscape-1.6.0-py3.10.egg\py4cytoscape\tables.py:502: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


Creating a node at each midpoint of the edge...
Column "KEGG_EDGE_SUBTYPES" has only 0 elements, but should have 1316
getting node positions...
getting edge information...
creating midpoint nodes...


c:\Users\fz8348\mambaforge\lib\site-packages\py4cytoscape-1.6.0-py3.10.egg\py4cytoscape\tables.py:502: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


deleting edges...
start collecting node IDs to connect...
Importing rno01100.xml


c:\Users\fz8348\mambaforge\lib\site-packages\py4cytoscape-1.6.0-py3.10.egg\py4cytoscape\tables.py:502: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


In [6]:
def importLayer(row: pd.Series) -> int:
    #print(row.array)
    #getKgml(row.array[1])
    r = requests.get("https://rest.kegg.jp/get/" + row.array[1] + "/kgml")
    kgml = row.array[1] + ".xml"
    f = open(kgml, "w")
    f.write(r.text)
    f.close()    
    print("Importing " + kgml)
    res = p4c.import_network_from_file(kgml)
    #time.sleep(3)
    xy = p4c.get_table_columns(table='node', columns=['KEGG_NODE_X', 'KEGG_NODE_Y'])
    xy.rename(columns={"KEGG_NODE_X": "x_location", "KEGG_NODE_Y": "y_location"}, inplace=True)
    p4c.load_table_data(xy, table_key_column="SUID")
    p4c.update_style_mapping(p4c.get_current_style(), p4c.map_visual_property("NODE_X_LOCATION", "x_location", "p"))
    p4c.update_style_mapping(p4c.get_current_style(), p4c.map_visual_property("NODE_Y_LOCATION", "y_location", "p"))
    p4c.fit_content()
    #print("Check point ...")
    # print(row.array[3])
    # print(type(row.array[3]))
    # if row.array[3] == "True":
    #     print("foo")
    if row.array[3]:
        print("Creating a node at each midpoint of the edge...")
        createMidnodes(res['networks'][0])
    return res['networks'][0]

In [83]:
def create3Dnetwork(networkLayers, stylexml):
    layerTable = pd.read_table(networkLayers, header=None, sep="\t")
    networkSUID = layerTable.apply(importLayer, axis=1)
    layerTable = pd.concat([layerTable, networkSUID], axis=1)
    nodetables = layerTable.apply(getNodeTableWithLayerinfo, axis=1)
    layeredNodes = getLayeredNodes(nodetables)
    edgetables = layerTable.apply(getEdgeTableWithLayerinfo, axis=1)
    layeredEdges = pd.concat(edgetables.tolist())
    # layeredNodes.to_csv("layeredNodes.csv")
    # layeredEdges.to_csv("layeredEdges.csv")
    p4c.commands_post('cy3d set renderer')
    suID = p4c.create_network_from_data_frames(layeredNodes.astype(str), layeredEdges.astype(str))
    setTransomicStyle(stylexml, suID)
    return suID

In [7]:
#networkSUID = 1162
def createMidnodes(networkSUID):
    et = p4c.get_table_columns(table='edge', network=networkSUID)
    print("getting node positions...")
    xyloc = p4c.get_table_columns(table='node', network=networkSUID, columns=['SUID', 'x_location', 'y_location'])
    print("getting edge information...")
    einfo = p4c.get_edge_info(et['SUID'].tolist(), network=networkSUID)
    einfodf = pd.DataFrame.from_dict(einfo)
    midxydf = pd.DataFrame.from_dict([getMidLoc(x, xyloc) for x in einfo])
    midNodeInfo = einfodf.join(midxydf)    
    midNodeInfo.rename(columns={"SUID": "orig_edge_SUID"}, inplace=True)
    print("creating midpoint nodes...")
    midNodes = p4c.add_cy_nodes(et['SUID'].apply(str).to_list())
    midNodeInfo['orig_edge_SUID'] = midNodeInfo['orig_edge_SUID'].apply(str)
    p4c.load_table_data(midNodeInfo, data_key_column="orig_edge_SUID", table="node", table_key_column="name")
    p4c.update_style_mapping(p4c.get_current_style(), p4c.map_visual_property("NODE_X_LOCATION", "x_location", "p"))
    p4c.update_style_mapping(p4c.get_current_style(), p4c.map_visual_property("NODE_Y_LOCATION", "y_location", "p"))
    print("deleting edges...")
    p4c.invert_edge_selection()
    p4c.delete_selected_edges()
    print("start collecting node IDs to connect...")
    midNodesName2suid =  pd.DataFrame.from_dict(midNodes)
    midNodeInfo = pd.merge(midNodeInfo, midNodesName2suid, left_on='orig_edge_SUID', right_on='name')
    src2mid = midNodeInfo[['source', 'SUID']]
    src2mid.rename(columns={"SUID": "target"}, inplace=True)
    mid2tgt = midNodeInfo[['SUID', 'target']]
    mid2tgt.rename(columns={"SUID": "source"}, inplace=True)
    res = p4c.cyrest_post("networks/" + str(networkSUID) + "/edges", body=src2mid.to_dict('records'))
    res = p4c.cyrest_post("networks/" + str(networkSUID) + "/edges", body=mid2tgt.to_dict('records'))

In [8]:
def getMidLoc(elem, xyloc):
    sourcexy = xyloc[xyloc['SUID'] == elem['source']]
    targetxy = xyloc[xyloc['SUID'] == elem['target']]
    midx_location = (int(sourcexy['x_location'].values[0]) + int(targetxy['x_location'].values[0])) / 2
    midy_location = (int(sourcexy['y_location'].values[0]) + int(targetxy['y_location'].values[0])) / 2
    #elem['x_location'] = midx_location
    #elem['y_location'] = midy_location
    #return elem
    return {'x_location': midx_location, 'y_location': midy_location}
    #return pd.Series({'x_location': midx_location, 'y_location': midy_location})

In [6]:
# def createMidnodes(networkSUID):
#     et = p4c.get_table_columns(table='edge', network=networkSUID)
#     print("getting node positions...")
#     xyloc = p4c.get_table_columns(table='node', network=networkSUID, columns=['SUID', 'x_location', 'y_location'])
#     print("getting edge information...")
#     einfo = p4c.get_edge_info(et['SUID'].tolist(), network=networkSUID)
#     print("checking mid point xy coordinates...")
#     midxydf = [getMidLoc(x, xyloc) for x in einfo]
#     midxydf = pd.concat(midxydf)
#     #midxydf = midxydf.rename(columns={"x_location": "x_location", "y_location": "y_location"})
#     midxydf = midxydf.drop_duplicates()
#     print("creating midnodes...")
#     p4c.create_network_from_data_frames(midxydf, network=networkSUID)
#     print("setting node positions...")
#     p4c.load_table_data(midxydf, table_key_column="SUID")
#     print("done")
    


In [64]:
networkSUID = 79748
et = p4c.get_table_columns(table='edge', network=networkSUID)
print("getting node positions...")
xyloc = p4c.get_table_columns(table='node', network=networkSUID, columns=['SUID', 'x_location', 'y_location'])
print("getting edge information...")
einfo = p4c.get_edge_info(et['SUID'].tolist(), network=networkSUID)
einfodf = pd.DataFrame.from_dict(einfo)


Column "KEGG_EDGE_SUBTYPES" has only 0 elements, but should have 1316
getting node positions...
getting edge information...


In [66]:
midxydf = [getMidLoc(x, xyloc) for x in einfo]

In [69]:
midxydf

[{'x_location': 2425.5, 'y_location': 1108.0},
 {'x_location': 2913.0, 'y_location': 1564.5},
 {'x_location': 3160.0, 'y_location': 1756.0},
 {'x_location': 2495.0, 'y_location': 1128.0},
 {'x_location': 2447.0, 'y_location': 1145.0},
 {'x_location': 3062.5, 'y_location': 1713.0},
 {'x_location': 2622.5, 'y_location': 1941.0},
 {'x_location': 2498.0, 'y_location': 1125.0},
 {'x_location': 2425.5, 'y_location': 1050.0},
 {'x_location': 2695.0, 'y_location': 1970.0},
 {'x_location': 3135.0, 'y_location': 1742.0},
 {'x_location': 2476.5, 'y_location': 1030.0},
 {'x_location': 2548.5, 'y_location': 1171.5},
 {'x_location': 3200.0, 'y_location': 1713.0},
 {'x_location': 3308.5, 'y_location': 1866.0},
 {'x_location': 2596.5, 'y_location': 1154.5},
 {'x_location': 2498.0, 'y_location': 1125.0},
 {'x_location': 3282.5, 'y_location': 1681.0},
 {'x_location': 3325.0, 'y_location': 2080.5},
 {'x_location': 2546.0, 'y_location': 1108.0},
 {'x_location': 2698.0, 'y_location': 1076.5},
 {'x_location

In [70]:
aaa = pd.DataFrame.from_dict(midxydf)

In [71]:
aaa

,x_location,y_location
0,2425.5,1108.0
1,2913.0,1564.5
2,3160.0,1756.0
3,2495.0,1128.0
4,2447.0,1145.0
...,...,...
1311,2043.0,1355.0
1312,2377.5,1125.0
1313,2563.0,1416.0
1314,2794.0,1416.0


In [73]:
bbb = aaa.join(einfodf)

In [74]:
bbb

,x_location,y_location,source,target,SUID,shared name,shared interaction,name,selected,interaction,KEGG_REACTION_TYPE,KEGG_REACTION_GENE
0,2425.5,1108.0,87229,87901,92161,rn:R01827,reversible,rn:R01827,False,reversible,reversible,rno:83688
1,2913.0,1564.5,87223,89725,90112,rn:R00582 rn:R12342 rn:R12344,reversible,rn:R00582 rn:R12342 rn:R12344,False,reversible,reversible,rno:304429
2,3160.0,1756.0,89740,89359,90115,rn:R00751,reversible,rn:R00751,False,reversible,reversible,rno:688286
3,2495.0,1128.0,87229,86137,92164,rn:R01827,reversible,rn:R01827,False,reversible,reversible,rno:83688
4,2447.0,1145.0,88861,86137,92167,rn:R01830,reversible,rn:R01830,False,reversible,reversible,rno:290685 rno:64524 rno:689374
...,...,...,...,...,...,...,...,...,...,...,...,...
1311,2043.0,1355.0,88735,88771,92152,rn:R01186,reversible,rn:R01186,False,reversible,reversible,rno:282636 rno:83523
1312,2377.5,1125.0,88861,87901,92155,rn:R01827,reversible,rn:R01827,False,reversible,reversible,rno:83688
1313,2563.0,1416.0,88858,87049,90106,rn:R01513,reversible,rn:R01513,False,reversible,reversible,rno:58835
1314,2794.0,1416.0,87223,87049,90109,rn:R04173,reversible,rn:R04173,False,reversible,reversible,rno:293820


In [72]:
einfodf

,source,target,SUID,shared name,shared interaction,name,selected,interaction,KEGG_REACTION_TYPE,KEGG_REACTION_GENE
0,87229,87901,92161,rn:R01827,reversible,rn:R01827,False,reversible,reversible,rno:83688
1,87223,89725,90112,rn:R00582 rn:R12342 rn:R12344,reversible,rn:R00582 rn:R12342 rn:R12344,False,reversible,reversible,rno:304429
2,89740,89359,90115,rn:R00751,reversible,rn:R00751,False,reversible,reversible,rno:688286
3,87229,86137,92164,rn:R01827,reversible,rn:R01827,False,reversible,reversible,rno:83688
4,88861,86137,92167,rn:R01830,reversible,rn:R01830,False,reversible,reversible,rno:290685 rno:64524 rno:689374
...,...,...,...,...,...,...,...,...,...,...
1311,88735,88771,92152,rn:R01186,reversible,rn:R01186,False,reversible,reversible,rno:282636 rno:83523
1312,88861,87901,92155,rn:R01827,reversible,rn:R01827,False,reversible,reversible,rno:83688
1313,88858,87049,90106,rn:R01513,reversible,rn:R01513,False,reversible,reversible,rno:58835
1314,87223,87049,90109,rn:R04173,reversible,rn:R04173,False,reversible,reversible,rno:293820


In [58]:
src2mid.to_dict('records')

[{'source': 8292, 'target': 26804},
 {'source': 7638, 'target': 25512},
 {'source': 8295, 'target': 26552},
 {'source': 7638, 'target': 26240},
 {'source': 9009, 'target': 26098},
 {'source': 9660, 'target': 25188},
 {'source': 8301, 'target': 24858},
 {'source': 9831, 'target': 25090},
 {'source': 8646, 'target': 25054},
 {'source': 7755, 'target': 25718},
 {'source': 7578, 'target': 25148},
 {'source': 9687, 'target': 24964},
 {'source': 8709, 'target': 27076},
 {'source': 7743, 'target': 26226},
 {'source': 7605, 'target': 26488},
 {'source': 9690, 'target': 27118},
 {'source': 7575, 'target': 25962},
 {'source': 7743, 'target': 25428},
 {'source': 7575, 'target': 24992},
 {'source': 9516, 'target': 26678},
 {'source': 7572, 'target': 25014},
 {'source': 9876, 'target': 26106},
 {'source': 7560, 'target': 24570},
 {'source': 9708, 'target': 25848},
 {'source': 7584, 'target': 24826},
 {'source': 9708, 'target': 26554},
 {'source': 10284, 'target': 25098},
 {'source': 9714, 'target':

In [42]:
midNodesName2suid =  pd.DataFrame.from_dict(midNodes)

In [49]:
midNodeInfo = pd.merge(midNodeInfo, midNodesName2suid, left_on='orig_edge_SUID', right_on='name')

In [50]:
src2mid = midNodeInfo[['source', 'SUID']]

In [51]:
src2mid

,source,SUID
0,8292,26804
1,7638,25512
2,8295,26552
3,7638,26240
4,9009,26098
...,...,...
1311,9651,26346
1312,8289,25738
1313,8946,25196
1314,8289,26282


In [36]:
midNodeInfo['orig_edge_SUID'] = midNodeInfo['orig_edge_SUID'].apply(str)

In [39]:
p4c.update_style_mapping(p4c.get_current_style(), p4c.map_visual_property("NODE_X_LOCATION", "x_location", "p"))
p4c.update_style_mapping(p4c.get_current_style(), p4c.map_visual_property("NODE_Y_LOCATION", "y_location", "p"))

''

In [38]:
p4c.load_table_data(midNodeInfo, data_key_column="orig_edge_SUID", table="node", table_key_column="name")

c:\Users\kozo2\mambaforge\lib\site-packages\py4cytoscape-1.6.0-py3.10.egg\py4cytoscape\tables.py:502: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  data_subset.dtypes.iteritems()]


'Success: Data loaded in defaultnode table'

In [25]:
midNodeInfo.rename(columns={"SUID": "orig_edge_SUID"}, inplace=True)

In [26]:
midNodeInfo

,source,target,orig_edge_SUID,shared name,shared interaction,name,selected,interaction,KEGG_REACTION_TYPE,KEGG_REACTION_GENE,x_location,y_location
0,8292,8295,12288,rn:R02843,reversible,rn:R02843,False,reversible,reversible,rno:24294 rno:353498 rno:500892,1198.0,2523.5
1,7638,9651,14337,rn:R11861,reversible,rn:R11861,False,reversible,reversible,rno:360994 rno:362196,3247.5,2500.5
2,8295,8298,12291,rn:R02834 rn:R02836,reversible,rn:R02834 rn:R02836,False,reversible,reversible,rno:25116 rno:25117,1198.0,2573.5
3,7638,8880,14340,rn:R11861,reversible,rn:R11861,False,reversible,reversible,rno:360994 rno:362196,3161.0,2483.0
4,9009,8301,12294,rn:R02213,reversible,rn:R02213,False,reversible,reversible,rno:24298,1132.0,2460.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1311,9651,9450,14328,rn:R00251,reversible,rn:R00251,False,reversible,reversible,rno:116684,3287.0,2425.5
1312,8289,9009,12282,rn:R02211,reversible,rn:R02211,False,reversible,reversible,rno:25146,1165.0,2422.0
1313,8946,9651,14331,rn:R02743,reversible,rn:R02743,False,reversible,reversible,rno:362368,3317.5,2500.5
1314,8289,8292,12285,rn:R03326,reversible,rn:R03326,False,reversible,reversible,rno:24298,1198.0,2460.0


In [31]:
et['SUID'].apply(str).to_list()

['12288',
 '14337',
 '12291',
 '14340',
 '12294',
 '14343',
 '12297',
 '14346',
 '12300',
 '14349',
 '12303',
 '14352',
 '12306',
 '14355',
 '12309',
 '14358',
 '12312',
 '14361',
 '12315',
 '14364',
 '12318',
 '14367',
 '12321',
 '14370',
 '12324',
 '14373',
 '12327',
 '14376',
 '12330',
 '14379',
 '12333',
 '14382',
 '12336',
 '14385',
 '12339',
 '14388',
 '12342',
 '14391',
 '12345',
 '14394',
 '12348',
 '14397',
 '12351',
 '14400',
 '12354',
 '14403',
 '12357',
 '14406',
 '12360',
 '14409',
 '12363',
 '14412',
 '12366',
 '14415',
 '12369',
 '14418',
 '12372',
 '14421',
 '12375',
 '14424',
 '12378',
 '14427',
 '12381',
 '14430',
 '12384',
 '14433',
 '12387',
 '14436',
 '12390',
 '14439',
 '12393',
 '14442',
 '12396',
 '14445',
 '12399',
 '14448',
 '12402',
 '14451',
 '12405',
 '14454',
 '12408',
 '14457',
 '12411',
 '14460',
 '12414',
 '14463',
 '12417',
 '14466',
 '12420',
 '14469',
 '12423',
 '14472',
 '12426',
 '14475',
 '12429',
 '14478',
 '12432',
 '14481',
 '12435',
 '14484',


In [16]:
einfodf = pd.DataFrame.from_dict(einfo)

In [17]:
einfodf

,source,target,SUID,shared name,shared interaction,name,selected,interaction,KEGG_REACTION_TYPE,KEGG_REACTION_GENE
0,8292,8295,12288,rn:R02843,reversible,rn:R02843,False,reversible,reversible,rno:24294 rno:353498 rno:500892
1,7638,9651,14337,rn:R11861,reversible,rn:R11861,False,reversible,reversible,rno:360994 rno:362196
2,8295,8298,12291,rn:R02834 rn:R02836,reversible,rn:R02834 rn:R02836,False,reversible,reversible,rno:25116 rno:25117
3,7638,8880,14340,rn:R11861,reversible,rn:R11861,False,reversible,reversible,rno:360994 rno:362196
4,9009,8301,12294,rn:R02213,reversible,rn:R02213,False,reversible,reversible,rno:24298
...,...,...,...,...,...,...,...,...,...,...
1311,9651,9450,14328,rn:R00251,reversible,rn:R00251,False,reversible,reversible,rno:116684
1312,8289,9009,12282,rn:R02211,reversible,rn:R02211,False,reversible,reversible,rno:25146
1313,8946,9651,14331,rn:R02743,reversible,rn:R02743,False,reversible,reversible,rno:362368
1314,8289,8292,12285,rn:R03326,reversible,rn:R03326,False,reversible,reversible,rno:24298


In [14]:
midxydf

x_location    1198.0
y_location    2523.5
x_location    3247.5
y_location    2500.5
x_location    1198.0
               ...  
y_location    2500.5
x_location    1198.0
y_location    2460.0
x_location    3263.5
y_location    2421.5
Length: 2632, dtype: float64

In [27]:
xyloc = p4c.get_table_columns(table='node', network=1162, columns=['SUID', 'x_location', 'y_location'])

In [39]:
(xyloc['SUID'] == 7638).value_counts()

False    3320
True        1
Name: SUID, dtype: int64

In [40]:
sourcexy = xyloc[xyloc['SUID'] == 7638]

In [31]:
xyloc['SUID'] == 8193

8193     True
8196    False
8199    False
8202    False
8205    False
        ...  
8178    False
8181    False
8184    False
8187    False
8190    False
Name: SUID, Length: 3321, dtype: bool

In [41]:
sourcexy

,SUID,x_location,y_location
7638,7638,3192,2483


In [45]:
sourcexy['x_location']

7638    3192
Name: x_location, dtype: object

In [48]:
sourcexy['x_location'].values[0]

'3192'

In [26]:
foo[3]

{'source': 7638,
 'target': 8880,
 'SUID': 14340,
 'shared name': 'rn:R11861',
 'shared interaction': 'reversible',
 'name': 'rn:R11861',
 'selected': False,
 'interaction': 'reversible',
 'KEGG_REACTION_TYPE': 'reversible',
 'KEGG_REACTION_GENE': 'rno:360994 rno:362196'}

In [14]:
et = p4c.get_table_columns(table='edge', network=1162)

Column "KEGG_EDGE_SUBTYPES" has only 0 elements, but should have 1316


In [18]:
et

,SUID,shared name,shared interaction,name,selected,interaction,KEGG_RELATION_TYPE,KEGG_REACTION_TYPE,KEGG_REACTION_GENE,KEGG_EDGE_COLOR,KEGG_EDGE_COLOR_BG
12288,12288,rn:R02843,reversible,rn:R02843,False,reversible,None,reversible,rno:24294 rno:353498 rno:500892,None,None
14337,14337,rn:R11861,reversible,rn:R11861,False,reversible,None,reversible,rno:360994 rno:362196,None,None
12291,12291,rn:R02834 rn:R02836,reversible,rn:R02834 rn:R02836,False,reversible,None,reversible,rno:25116 rno:25117,None,None
14340,14340,rn:R11861,reversible,rn:R11861,False,reversible,None,reversible,rno:360994 rno:362196,None,None
12294,12294,rn:R02213,reversible,rn:R02213,False,reversible,None,reversible,rno:24298,None,None
...,...,...,...,...,...,...,...,...,...,...,...
14328,14328,rn:R00251,reversible,rn:R00251,False,reversible,None,reversible,rno:116684,None,None
12282,12282,rn:R02211,reversible,rn:R02211,False,reversible,None,reversible,rno:25146,None,None
14331,14331,rn:R02743,reversible,rn:R02743,False,reversible,None,reversible,rno:362368,None,None
12285,12285,rn:R03326,reversible,rn:R03326,False,reversible,None,reversible,rno:24298,None,None


In [20]:
et['SUID'].tolist()

[12288,
 14337,
 12291,
 14340,
 12294,
 14343,
 12297,
 14346,
 12300,
 14349,
 12303,
 14352,
 12306,
 14355,
 12309,
 14358,
 12312,
 14361,
 12315,
 14364,
 12318,
 14367,
 12321,
 14370,
 12324,
 14373,
 12327,
 14376,
 12330,
 14379,
 12333,
 14382,
 12336,
 14385,
 12339,
 14388,
 12342,
 14391,
 12345,
 14394,
 12348,
 14397,
 12351,
 14400,
 12354,
 14403,
 12357,
 14406,
 12360,
 14409,
 12363,
 14412,
 12366,
 14415,
 12369,
 14418,
 12372,
 14421,
 12375,
 14424,
 12378,
 14427,
 12381,
 14430,
 12384,
 14433,
 12387,
 14436,
 12390,
 14439,
 12393,
 14442,
 12396,
 14445,
 12399,
 14448,
 12402,
 14451,
 12405,
 14454,
 12408,
 14457,
 12411,
 14460,
 12414,
 14463,
 12417,
 14466,
 12420,
 14469,
 12423,
 14472,
 12426,
 14475,
 12429,
 14478,
 12432,
 14481,
 12435,
 14484,
 12438,
 14487,
 12441,
 14490,
 12444,
 14493,
 12447,
 14496,
 12450,
 14499,
 12453,
 14502,
 12456,
 14505,
 12459,
 14508,
 12462,
 14511,
 12465,
 14514,
 12468,
 14517,
 12471,
 14520,
 12474,


In [22]:
foo = p4c.get_edge_info(et['SUID'].tolist(), network=1162)

In [25]:
len(foo)

1316

In [54]:
# et = p4c.get_table_columns(table='edge', network=213631)

Column "KEGG_EDGE_SUBTYPES" has only 0 elements, but should have 1312


In [60]:
# xyloc = p4c.get_table_columns(table='node', network=213631, columns=['SUID', 'x_location', 'y_location'])

In [61]:
# xyloc

,SUID,x_location,y_location
221187,221187,1746,1004
221184,221184,871,971
221190,221190,2140,1120
221193,221193,1892,748
221199,221199,3201,1027
...,...,...,...
221169,221169,1800,1004
221175,221175,1295,971
221172,221172,1344,971
221178,221178,1247,971


In [67]:
# int(xyloc[xyloc['SUID'] == 221187]['x_location'].values[0])

1746

In [58]:
# einfo = p4c.get_edge_info(et['SUID'].tolist(), network=213631)

In [59]:
# einfo

[{'source': 223332,
  'target': 223335,
  'SUID': 227331,
  'shared name': 'rn:R04867',
  'shared interaction': 'reversible',
  'name': 'rn:R04867',
  'selected': False,
  'interaction': 'reversible',
  'KEGG_REACTION_TYPE': 'reversible',
  'KEGG_REACTION_GENE': 'rno:100359816 rno:100361710 rno:100362634 rno:102550344 rno:289372 rno:297333 rno:299521 rno:302553 rno:303547 rno:310638 rno:311968 rno:312299 rno:313678 rno:315606 rno:316013 rno:361798 rno:362078 rno:362831 rno:364785 rno:365155 rno:498295 rno:502710 rno:689883'},
 {'source': 220059,
  'target': 220062,
  'SUID': 225282,
  'shared name': 'rn:R05976',
  'shared interaction': 'reversible',
  'name': 'rn:R05976',
  'selected': False,
  'interaction': 'reversible',
  'KEGG_REACTION_TYPE': 'reversible',
  'KEGG_REACTION_GENE': 'rno:192275 rno:25596 rno:290783 rno:313648 rno:363160 rno:500972 rno:64701'},
 {'source': 223329,
  'target': 223332,
  'SUID': 227328,
  'shared name': 'rn:R04866',
  'shared interaction': 'reversible',


In [87]:
def getNodeTableWithLayerinfo(row: pd.Series) -> pd.DataFrame:
    nt = p4c.get_table_columns(table="node", network=row.array[4])
    nt['z_location'] = row.array[2]
    nt['LAYER_INDEX'] = row.array[0]
    return nt

In [88]:
def getLayeredNodes(nodetables: pd.DataFrame) -> pd.DataFrame:
    nodetable3d = pd.concat(nodetables.array)
    return nodetable3d.rename(columns={"SUID": "id"})

In [89]:
def getEdgeTableWithLayerinfo(row: pd.Series) -> pd.DataFrame:
    et = p4c.get_table_columns(table="edge", network=row.array[4])
    #return et
    print("Getting edge info. This function is kinda slow...")
    ei = p4c.get_edge_info(et["SUID"].tolist(), network=row.array[4])
    print("Finished getting edge info.")
    et['source'] = list(map(lambda x: x['source'], ei))
    et['target'] = list(map(lambda x: x['target'], ei))
    et['LAYER_INDEX'] = row.array[0]
    return et

In [90]:
def setTransomicStyle(xml, suid) -> None:
    #xml is must be a local file path
    stylename = p4c.import_visual_styles(xml)
    res = p4c.set_visual_style(stylename[0], network = suid)
    print(stylename[0] + " " + res['message'])